# Unity ML-Agents Toolkit
## Environment Basics
This notebook contains a walkthrough of the basic functions of the Python API for the Unity ML-Agents toolkit. For instructions on building a Unity environment, see [here](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Getting-Started-with-Balance-Ball.md).

### 1. Set environment parameters

Be sure to set `env_name` to the name of the Unity environment file you want to launch. Ensure that the environment build is in the `python/` directory.

In [3]:
env_name = "Car\Car"  # Name of the Unity environment binary to launch
train_mode = True  # Whether to run the environment in training or inference mode

### 2. Load dependencies

The following loads the necessary dependencies and checks the Python version (at runtime). ML-Agents Toolkit (v0.3 onwards) requires Python 3.

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import sys
import cv2
import tensorflow as tf
from numpy import newaxis
from sklearn.utils import shuffle
from unityagents import UnityEnvironment
import yolo as yl
from PIL import Image

%matplotlib inline

print("Python version:")
print(sys.version)

yolo=yl.YOLO()

# check Python version
if (sys.version_info[0] < 3):
    raise Exception("ERROR: ML-Agents Toolkit (v0.3 onwards) requires Python 3")

Python version:
3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]


RuntimeError: Attempted to use a closed Session.

### 3. Start the environment
`UnityEnvironment` launches and begins communication with the environment when instantiated.

Environments contain _brains_ which are responsible for deciding the actions of their associated _agents_. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
env = UnityEnvironment(file_name=env_name)

# Examine environment parameters
print(str(env))

# Set the default brain to work with
default_brain = env.brain_names[0]
brain = env.brains[default_brain]

# Reset the environment
env_info = env.reset(train_mode=train_mode)[default_brain]
img=env_info.visual_observations[0][0]
#img = input('Input your image')
#image = Image.open(img)
image=img
r_image = yolo.detect_image(image)
r_image.show()
env.close()
yolo.close_session()
return

### 4. Examine the observation and state spaces
We can reset the environment to be provided with an initial set of observations and states for all the agents within the environment. In ML-Agents, _states_ refer to a vector of variables corresponding to relevant aspects of the environment for an agent. Likewise, _observations_ refer to a set of relevant pixel-wise visuals for an agent.

In [ ]:
# Reset the environment
env_info = env.reset(train_mode=train_mode)[default_brain]

img=env_info.visual_observations[0][0]
#showEdges(img)
#plt.imshow(aaa[0,:,:,:])

In [ ]:
def edges(img, thr1=100, thr2=150):
    imgBlur = cv2.GaussianBlur(img, (5, 5), 0)
    edges = cv2.Canny(np.uint8(imgBlur*255),thr1,thr2)/255
    return edges.reshape(-1,*edges.shape,1)

def showEdges(img, thr1=100, thr2=150):
    imgBlur = cv2.GaussianBlur(img, (5, 5), 0)
    edges = cv2.Canny(np.uint8(imgBlur*255),thr1,thr2)/255
    plt.subplots(figsize=(15,15))
    plt.subplot(131), plt.xticks([]), plt.yticks([]), plt.title('Original Image')
    plt.imshow(img)
    plt.subplot(132), plt.xticks([]), plt.yticks([]), plt.title('Blurred Image')
    plt.imshow(imgBlur)
    plt.subplot(133), plt.xticks([]), plt.yticks([]), plt.title('Edges')
    plt.imshow(edges, cmap='gray')
    plt.show()

In [ ]:
##Object detection
import os
from utils import label_map_util
from utils import visualization_utils as vis_util
import pickle

# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
        
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

#---------------------------------------------------------------------




env_info = env.reset(train_mode=False)[default_brain] # reset the environment

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        
        image_np = (env_info.visual_observations[0][0]*255).astype(np.uint8)
        (boxes, scores, classes, num) = sess.run(
                      [detection_boxes, detection_scores, detection_classes, num_detections],
                      feed_dict={image_tensor: image_np[np.newaxis, :]})
        # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index, 
                min_score_thresh = scores[0][0]-1e-9,
                use_normalized_coordinates=True,
                line_thickness=2)
        plt.figure(figsize=(9,6))
        plt.xticks([]), plt.yticks([])
        plt.imshow(image_np)
        plt.show()

### 5. Take random actions in the environment
Once we restart an environment, we can step the environment forward and provide actions to all of the agents within the environment. Here we simply choose random actions based on the `action_space_type` of the default brain. 

Once this cell is executed, 10 messages will be printed that detail how much reward will be accumulated for the next 10 episodes. The Unity environment will then pause, waiting for further signals telling it what to do next. Thus, not seeing any animation is expected when running this cell.

In [ ]:
for episode in range(1):
    env_info = env.reset(train_mode=train_mode)[default_brain]
    done = False
    episode_rewards = 0
    while not done:
        if brain.vector_action_space_type == 'continuous':
            env_info = env.step([1,0,0])[default_brain]
        else:
            env_info = env.step([1])[default_brain]
        episode_rewards += env_info.rewards[0]
        print("Agent state looks like: \n{}".format(env_info.vector_observations[0]))
        done = env_info.local_done[0]
    print("Total reward this episode: {}".format(episode_rewards))

### 6. Close the environment when finished
When we are finished using an environment, we can close it with the function below.

In [ ]:
env.close()
yolo.close_session()